# Notebook 01: Data Exploration

## Project Overview

This notebook explores the raw Apple options dataset to understand its structure and prepare for machine learning.

**Objectives:**
- Load and clean the raw CSV data
- Inspect data types and missing values
- Analyze distributions of key features (underlying price, strike, time to expiry, implied volatility, option prices)
- Identify outliers and data quality issues
- Understand why Greeks should not be used as ML features

**Dataset:** Apple (AAPL) call option quotes containing market prices, implied volatilities, and Greeks.

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

PROJECT_ROOT = Path("..").resolve()
DATA_RAW_DIR = PROJECT_ROOT / "data" / "raw"
DATA_INTERIM_DIR = PROJECT_ROOT / "data" / "interim"
FIGS_DIR = PROJECT_ROOT / "figs"

DATA_INTERIM_DIR.mkdir(parents=True, exist_ok=True)
FIGS_DIR.mkdir(parents=True, exist_ok=True)

RAW_FILE = DATA_RAW_DIR / "AAPL_DATA.csv"

In [ ]:
df_raw = pd.read_csv(RAW_FILE, low_memory=False)
print(f"Dataset: {df_raw.shape[0]:,} rows x {df_raw.shape[1]} columns")
df_raw.head()

## 2. Data Cleaning and Type Conversion

### 2.1 Clean Column Names

The raw column names contain square brackets and extra spaces (e.g., `[C_LAST]`, ` [STRIKE]`).  
We standardize them for easier handling.

In [ ]:
df_raw.columns = [col.strip().strip("[]") for col in df_raw.columns]
df_raw.columns.tolist()

### 2.2 Convert Text Columns to Numbers

Many numeric columns are loaded as text ('object' type). We convert them to float.

In [ ]:
numeric_cols = [
    "C_DELTA", "C_GAMMA", "C_VEGA", "C_THETA", "C_RHO",
    "C_IV", "C_VOLUME", "C_LAST", "C_BID", "C_ASK",
    "P_BID", "P_ASK", "P_LAST",
    "P_DELTA", "P_GAMMA", "P_VEGA", "P_THETA", "P_RHO", "P_IV", "P_VOLUME"
]

for col in numeric_cols:
    if col in df_raw.columns:
        df_raw[col] = pd.to_numeric(df_raw[col], errors="coerce")

df_raw.dtypes

### 2.3 Check for Missing Values

After conversion, some columns may contain NaN (missing) values.  
This is normal for option data: some quotes lack implied volatility or volume information.

In [ ]:
missing_counts = df_raw[numeric_cols].isna().sum()
missing_pct = (missing_counts / len(df_raw) * 100).round(2)

missing_df = pd.DataFrame({
    "Missing Count": missing_counts,
    "Missing %": missing_pct
}).sort_values("Missing Count", ascending=False)

missing_df[missing_df["Missing Count"] > 0]

## 3. Descriptive Statistics

We compute summary statistics for the main features to understand their ranges and distributions.

In [ ]:
key_cols = ["UNDERLYING_LAST", "STRIKE", "DTE", "C_IV", "C_LAST", "STRIKE_DISTANCE_PCT"]
df_raw[key_cols].describe()

**Interpretation:**

- **UNDERLYING_LAST**: Apple stock price ranges from 90 to 506 USD, indicating the dataset spans multiple market periods
- **STRIKE**: Strikes range from 2.5 to 1000 USD, covering deep ITM to far OTM options
- **DTE**: Maturities range from 0 (expiration day) to 890 days (long-dated options)
- **C_IV**: Implied volatility varies widely (see max values), indicating presence of outliers
- **C_LAST**: Call prices range from near 0 to very high values (deep ITM options)

## 4. Distribution Analysis

### 4.1 Underlying Price Distribution

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df_raw["UNDERLYING_LAST"], kde=True, bins=50, color="steelblue")
plt.title("Distribution of Apple Stock Price (UNDERLYING_LAST)")
plt.xlabel("Stock Price (USD)")
plt.ylabel("Frequency")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "dist_underlying_last.png", dpi=100)
plt.show()

**Interpretation:**  
The distribution shows multiple clusters, reflecting different market regimes for Apple stock over time.  
Most quotes were collected when the stock traded between 100-200 USD.

### 4.2 Strike Distribution

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df_raw["STRIKE"], kde=True, bins=80, color="darkorange")
plt.title("Distribution of Strike Prices")
plt.xlabel("Strike (USD)")
plt.ylabel("Frequency")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "dist_strike.png", dpi=100)
plt.show()

**Interpretation:**  
The distribution is right-skewed with most strikes between 50-250 USD.  
This aligns with the underlying price distribution.

### 4.3 Days to Expiry (DTE) Distribution

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df_raw["DTE"], kde=True, bins=80, color="green")
plt.title("Distribution of Days to Expiry (DTE)")
plt.xlabel("Days")
plt.ylabel("Frequency")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "dist_DTE.png", dpi=100)
plt.show()

**Interpretation:**  
The dataset is heavily dominated by short-term options (0-30 days).  
This is typical for equity options where weekly contracts are very popular.  
Long-dated options (LEAPS, > 365 days) are present but much less frequent.

### 4.4 Implied Volatility (C_IV) Analysis

Implied volatility is a critical input for option pricing.  
We check for outliers using both raw scale and log scale.

In [ ]:
c_iv_clean = df_raw["C_IV"].dropna()
print("C_IV Percentiles:")
print(c_iv_clean.quantile([0.90, 0.95, 0.99, 0.999]))

In [ ]:
c_iv_zoom = c_iv_clean[(c_iv_clean >= 0) & (c_iv_clean <= 1)]

plt.figure(figsize=(10, 5))
sns.histplot(c_iv_zoom, bins=80, kde=True, stat="density", alpha=0.6, color="purple")
plt.title("Distribution of Implied Volatility (C_IV) - Zoomed to [0, 1]")
plt.xlabel("Implied Volatility")
plt.ylabel("Density")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "dist_C_IV_zoom_0_1.png", dpi=100)
plt.show()

**Interpretation:**  
Most implied volatilities are between 0.2 and 0.4 (20-40%), which is typical for Apple options.  
The peak around 0.25-0.30 represents normal market conditions.

In [ ]:
plt.figure(figsize=(12, 3))
sns.boxplot(x=c_iv_clean, color="skyblue")
plt.title("Boxplot of Implied Volatility (C_IV) - Shows Extreme Outliers")
plt.xlabel("Implied Volatility")
plt.tight_layout()
plt.savefig(FIGS_DIR / "boxplot_C_IV.png", dpi=100)
plt.show()

**Interpretation:**  
The boxplot reveals many extreme outliers (IV > 5 or even > 50).  
These likely represent:
- Illiquid options with stale quotes
- Deep out-of-the-money options with unrealistic IVs
- Data errors

**Action:** These outliers will be filtered in Notebook 02 (preprocessing).

### 4.5 Call Price (C_LAST) Analysis

C_LAST is our target variable (what we want to predict).

In [ ]:
c_last_clean = df_raw["C_LAST"].dropna()
c_last_clean.describe()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(c_last_clean, bins=100, stat="density", alpha=0.6, color="coral")
plt.title("Distribution of Call Prices (C_LAST) - Raw Scale")
plt.xlabel("Call Price (USD)")
plt.ylabel("Density")
plt.xlim(0, 100)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "dist_C_LAST_raw.png", dpi=100)
plt.show()

**Interpretation:**  
Call prices are extremely right-skewed.  
Many options trade near zero (far OTM), while deep ITM options can exceed 100 USD.  
This heavy-tailed distribution motivates using a log transformation for modeling.

In [ ]:
c_last_pos = c_last_clean[c_last_clean > 0]

plt.figure(figsize=(10, 5))
sns.histplot(np.log(c_last_pos), bins=80, stat="density", alpha=0.6, kde=True, color="teal")
plt.title("Distribution of log(C_LAST) - Much More Symmetric")
plt.xlabel("log(Call Price)")
plt.ylabel("Density")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "dist_log_C_LAST.png", dpi=100)
plt.show()

## 5. Feature Engineering: Log-Moneyness

**Moneyness** measures how far the strike is from the stock price.  
We use **log-moneyness** = log(S/K), a standard measure in finance:
- log(S/K) = 0 → At-the-money (ATM)
- log(S/K) > 0 → In-the-money (ITM)
- log(S/K) < 0 → Out-of-the-money (OTM)

In [ ]:
df_raw["LOG_MONEYNESS"] = np.log(df_raw["UNDERLYING_LAST"] / df_raw["STRIKE"])
df_raw["LOG_MONEYNESS"].describe()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df_raw["LOG_MONEYNESS"].dropna(), bins=80, stat="density", alpha=0.6, kde=True, color="darkblue")
plt.axvline(0, color='red', linestyle='--', label='ATM (log(S/K)=0)')
plt.title("Distribution of Log-Moneyness = log(S/K)")
plt.xlabel("log(S/K)")
plt.ylabel("Density")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIGS_DIR / "dist_log_moneyness.png", dpi=100)
plt.show()

**Interpretation:**  
The distribution is centered near 0 (ATM options dominate).  
The right tail (ITM) is longer than the left tail (OTM), reflecting more deep ITM contracts.

## 6. Volatility Smile Analysis

A key feature of option markets is the **volatility smile**: implied volatility is not constant across strikes.  
We visualize IV vs log-moneyness, colored by maturity.

In [ ]:
mask = (
    (df_raw["C_IV"] > 0) &
    (df_raw["C_IV"] < 1.2) &
    df_raw["LOG_MONEYNESS"].notna() &
    df_raw["DTE"].notna()
)

df_plot = df_raw.loc[mask, ["C_IV", "LOG_MONEYNESS", "DTE"]].sample(20000, random_state=42)

plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    df_plot["LOG_MONEYNESS"],
    df_plot["C_IV"],
    c=df_plot["DTE"],
    cmap="viridis",
    s=12,
    alpha=0.45
)

cbar = plt.colorbar(scatter)
cbar.set_label("Days to Expiry (DTE)")

plt.title("Volatility Smile: IV vs Log-Moneyness (colored by maturity)")
plt.xlabel("log(S/K)")
plt.ylabel("Implied Volatility")
plt.ylim(0, 1.0)
plt.grid(alpha=0.25)
plt.tight_layout()
plt.savefig(FIGS_DIR / "smile_scatter_colored.png", dpi=100)
plt.show()

**Interpretation:**  
The scatter plot shows a clear **volatility smile**:
- IV is lowest near log(S/K) = 0 (ATM options)
- IV increases for both ITM and OTM options
- Short-term options (dark purple) have steeper smiles
- Long-term options (yellow/green) have flatter smiles

This structure is typical in equity option markets and confirms our data is realistic.

## 7. Why Greeks Should NOT Be Used as ML Features

### 7.1 What are Greeks?

Greeks (Delta, Gamma, Vega, Theta, Rho) measure the sensitivity of option prices to various factors.  
They are **calculated using pricing models** (usually Black-Scholes) based on:
- Stock price (S)
- Strike (K)
- Time to expiry (T)
- Implied volatility (σ)
- Interest rate (r)

### 7.2 The Problem: Data Leakage

Greeks are **NOT independent features**.  
They are derived from the option price (or a theoretical model).  

**Using Greeks as ML features creates data leakage because:**
1. Greeks encode the same information as S, K, T, σ (our features)
2. They are model-dependent (different vendors use different formulas)
3. In reality, we cannot know the Greeks before we know the price

**Conclusion:** Greeks are useful for analysis and validation, but should be **excluded from ML models**.

### 7.3 Quick Visual Check of Greeks

We visualize Delta and Vega distributions to understand their behavior.

In [ ]:
delta = df_raw["C_DELTA"].dropna()
vega = df_raw["C_VEGA"].dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(delta, bins=80, ax=axes[0], alpha=0.6, kde=True, color="blue")
axes[0].set_title("Distribution of Delta (C_DELTA)")
axes[0].set_xlabel("Delta")
axes[0].set_ylabel("Frequency")
axes[0].grid(alpha=0.3)

sns.histplot(vega, bins=80, ax=axes[1], alpha=0.6, kde=True, color="red")
axes[1].set_title("Distribution of Vega (C_VEGA)")
axes[1].set_xlabel("Vega")
axes[1].set_ylabel("Frequency")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGS_DIR / "greeks_delta_vega.png", dpi=100)
plt.show()

**Interpretation:**
- **Delta** has a U-shaped distribution (many near 0 and 1)
- **Vega** is right-skewed (most options have low vega)

These patterns reflect option moneyness, but add no new information beyond what we already have in (S, K, T, σ).

## 8. Correlation Analysis

In [ ]:
cols_corr = [
    "UNDERLYING_LAST",
    "STRIKE",
    "DTE",
    "LOG_MONEYNESS",
    "C_IV",
    "C_LAST"
]

corr_matrix = df_raw[cols_corr].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    linewidths=0.5,
    cbar_kws={"label": "Correlation"}
)
plt.title("Correlation Heatmap (Key Financial Variables)")
plt.tight_layout()
plt.savefig(FIGS_DIR / "correlation_heatmap.png", dpi=100)
plt.show()

**Key Observations:**
- **C_LAST** (target) correlates strongly with UNDERLYING_LAST (intrinsic value effect)
- **LOG_MONEYNESS** has negative correlation with STRIKE (by construction)
- **C_IV** shows moderate correlation with other features
- No extreme multicollinearity issues detected

## 9. Summary and Next Steps

### Key Findings:

1. **Dataset size:** 1,015,352 observations with 33 columns
2. **Missing values:** C_IV has ~7% missing, will be handled in preprocessing
3. **Outliers:** Extreme IVs (>5) and prices exist, will be filtered
4. **Distributions:** Most variables are right-skewed, log transformations help
5. **Volatility smile:** Clear pattern visible, confirms realistic option data

### Features Selected for Modeling:

**Input features:**
- UNDERLYING_LAST (stock price)
- STRIKE (option strike)
- DTE (days to expiry)
- LOG_MONEYNESS = log(S/K)
- C_IV (implied volatility)

**Target variable:**
- C_LAST (call option price)

**Excluded:**
- Greeks (C_DELTA, C_GAMMA, etc.) → data leakage
- Volume, bid/ask → not relevant for price prediction